# Modeling: Fit Logistic Regression

In [124]:
# Import  modules
## Data Imports
import numpy as np
import pandas as pd 
## Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [125]:
# Import functions
## Display
from IPython.display import display
## Classification and Regression tools
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import metrics
## Random
from numpy.random import choice
from numpy.random import seed
seed_number = 2015

## Get the data

In [126]:
# Import files
data_path = 'C:/Repositories/Titanic/data/'
types = {
    'Age':       np.float16,
    'Cabin':     object,
    'Embarked':  object,
    'Fare':      np.float16,
    'Name':      object,
    'Parch':     np.uint8,
    'Pclass':    np.uint8,
    'Sex':       object, #pd.category,
    'SibSp':     np.uint8,
    'Survived':  np.uint8,
    'Ticket':    object,
    'Title':     object,
    'FirstName': object,
    'LastName':  object,
    'Deck':      object
        }
titanic_train = pd.read_csv(data_path + 'train_DPP.csv', index_col='PassengerId', dtype=types)
titanic_test  = pd.read_csv(data_path + 'test_DPP.csv',  index_col='PassengerId', dtype=types)
display(titanic_train.tail())
#display(titanic_test.head())

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Title,FirstName,LastName,Deck
PassengerId,,,,,,,,,,,,,,,
887,27,NaN,S,13.000000,"Montvila, Rev. Juozas",0,2,male,0,0,211536,Rev.,NaN,Montvil,NaN
888,19,B42,S,30.000000,"Graham, Miss. Margaret Edith",0,1,female,0,1,112053,Miss.,Margaret,Graha,B
889,NaN,NaN,S,23.453125,"Johnston, Miss. Catherine Helen ""Carrie""",2,3,female,1,0,W./C. 6607,Miss.,Catherine,Johnsto,NaN
890,26,C148,C,30.000000,"Behr, Mr. Karl Howell",0,1,male,0,1,111369,Mr.,Karl,Beh,C
891,32,NaN,Q,7.750000,"Dooley, Mr. Patrick",0,3,male,0,0,370376,Mr.,Patrick,Doole,NaN


In [127]:
# Check columns type
titanic_train.dtypes

Age          float16
Cabin         object
Embarked      object
Fare         float16
Name          object
Parch          uint8
Pclass         uint8
Sex           object
SibSp          uint8
Survived       uint8
Ticket        object
Title         object
FirstName     object
LastName      object
Deck          object
dtype: object

## Data Preparation

### Set variables type

In [128]:
## Convert to categorical variables
categorical_variables = ["Sex","Survived"]
for var in categorical_variables:
    titanic_train[var] = titanic_train[var].astype('category')

categorical_variables.remove("Survived")
for var in categorical_variables:
    titanic_test[var]  = titanic_test[var].astype('category')

In [129]:
titanic_train.dtypes

Age           float16
Cabin          object
Embarked       object
Fare          float16
Name           object
Parch           uint8
Pclass          uint8
Sex          category
SibSp           uint8
Survived     category
Ticket         object
Title          object
FirstName      object
LastName       object
Deck           object
dtype: object

### Split the data

* 70% train set
* 30% test set

In [130]:
X_train,X_test,y_train,y_test = train_test_split(titanic_train.drop("Survived",1), # X
                                                 titanic_train.loc[:,"Survived"],  # y
                                                 test_size = 0.3,
                                                 random_state = seed_number)
# Flatten array converstion
# pandas.core.series.Series --> numpy.ndarray
#y_train = np.ravel(y_train)

## Fit Logistic Regression Model

### Gender Model

In [131]:
mdl_log_gender = LogisticRegression()
mdl_log_gender.fit(X_train[["Sex"]],y_train)

ValueError: could not convert string to float: 'female'

In [ ]:
#mdl_model.score(X_train,y_train)